In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects

import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization


from statsmodels.api import qqplot #for qq plot
from statsmodels.stats.diagnostic import het_breuschpagan,linear_rainbow  #variance equality and linearity tests
from scipy.stats import shapiro #for checking normality

ModuleNotFoundError: No module named 'linearmodels'

In [2]:
!pip install linearmodels

In [3]:
from linearmodels import PanelOLS

ModuleNotFoundError: No module named 'linearmodels'

In [4]:
conda install linearmodels


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - linearmodels

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [ ]:
data = pd.read_excel('data.xlsx')
data.head()

In [103]:
data = data.fillna(method='bfill')

In [104]:
data['total_debt']= data['total_credit_debt']+data['total_debitory_debt']
data['illegal_acts']=data['illegal_acts']/data['population']
data['passenger_turnover']=data['passenger_turnover']/data['population']
data['passenger_turnover']=data['passenger_turnover']/data['population']
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 19 columns):
marzname                            110 non-null object
marz                                110 non-null int64
year                                110 non-null int64
radianceoverpixels                  110 non-null float64
summedradiance                      110 non-null float64
population                          110 non-null float64
radpop_scaled                       110 non-null float64
average_expenditure                 110 non-null float64
average_tot_income                  110 non-null float64
consumption_per_adult_equivalent    110 non-null float64
headyears_educated                  110 non-null float64
construction_mln_amd                110 non-null float64
hospital_visits_per_Capita          110 non-null float64
illegal_acts                        110 non-null float64
total_credit_debt                   110 non-null float64
total_debitory_debt                 110 non-n

In [105]:
data= data.drop(['total_credit_debt', 'total_debitory_debt', 'construction_mln_amd'], axis=1)


In [106]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

year = pd.Categorical(data.year)
data = data.set_index(['marz','year'])
data['year'] = year
print(data.head())



          marzname  radianceoverpixels  summedradiance  population  \
marz year                                                            
1    2004  Armavir                 4.5          8469.5    256072.9   
     2005  Armavir                 4.6          8662.0    256143.6   
     2006  Armavir                 5.1          9764.5    256214.3   
     2007  Armavir                 6.0         11499.5    256285.1   
     2008  Armavir                 7.3         13914.0    256355.8   

           radpop_scaled  average_expenditure  average_tot_income  \
marz year                                                           
1    2004           33.1              67657.8             69964.1   
     2005           33.8              64208.7             76897.8   
     2006           38.1              75313.5             88649.5   
     2007           44.9             101420.1             80827.9   
     2008           54.3             108806.4            115331.6   

           consumption_pe

## Basic Regression on Panel Data

In [107]:
#PooledOLS is just plain OLS that understands that various panel data structures.
#It is useful as a base model. 
#Here the log wage is modeled using all of the variables and time dummies.

In [131]:

from linearmodels.panel import PooledOLS
import statsmodels.api as sm
exog_vars = ['radianceoverpixels', 'summedradiance', 'population', 'radpop_scaled', 'headyears_educated', 'hospital_visits_per_Capita', 'illegal_acts', 'year']
exog = sm.add_constant(data[exog_vars])
mod = PooledOLS(np.log(data.consumption_per_adult_equivalent), exog)
pooled_res = mod.fit()
print(pooled_res)

                                 PooledOLS Estimation Summary                                 
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.9756
Estimator:                                PooledOLS   R-squared (Between):              0.7811
No. Observations:                               110   R-squared (Within):               0.9807
Date:                              Sat, Apr 25 2020   R-squared (Overall):              0.9756
Time:                                      12:02:16   Log-likelihood                    126.06
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      232.32
Entities:                                        11   P-value                           0.0000
Avg Obs:                                    10.0000   Distribution:                   F(16,93)
Min Obs:                                    10.000

## Random Effects Model

In [109]:
#When modeling panel data it is common to consider models beyond what OLS will efficiently estimate. 
#The most common are error component models which add an additional term to the standard OLS model,
# y(it)=x(it)β+α(i)+ϵ(it)
#where α(i) affects all values of entity i. When the α(i) are uncorrelated with the regressors in x(it), 
#a random effects model can be used to efficiently estimate parameters of this model.

In [110]:
#The random effects model is virtually identical to the pooled OLS model except
#that is accounts for the structure of the model and so is more efficient. 
#Random effects uses a quasi-demeaning strategy which subtracts the time average 
#fo the within entity values to account for the common shock.

In [111]:
from linearmodels.panel import RandomEffects
mod = RandomEffects(np.log(data.consumption_per_adult_equivalent), exog)
re_res = mod.fit()
print(re_res)

                               RandomEffects Estimation Summary                               
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.9756
Estimator:                            RandomEffects   R-squared (Between):              0.7811
No. Observations:                               110   R-squared (Within):               0.9807
Date:                              Fri, Apr 24 2020   R-squared (Overall):              0.9756
Time:                                      22:15:52   Log-likelihood                    126.06
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      232.32
Entities:                                        11   P-value                           0.0000
Avg Obs:                                    10.0000   Distribution:                   F(16,93)
Min Obs:                                    10.000

In [112]:
#The model fit is fairly similar, although the return to experience has changed substantially, 
#as has its significance. This is partially explainable by the inclusion of the year dummies 
#which will fit the trend in experience and so only the cross-sectional differences matter. 
#The quasi-differencing in the random effects estimator depends on a quantity that depends 
#on the relative variance of the idiosyncratic shock and the common shock. 
#This can be accessed using

In [113]:
re_res.variance_decomposition

Effects                   0.000000
Residual                  0.005603
Percent due to Effects    0.000000
Name: Variance Decomposition, dtype: float64

In [114]:
re_res.theta.head()

,theta
marz,
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0


In [115]:
#The coefficient θi determines how much demeaning takes place. 
#When this value is 1, the RE model reduces to the pooled model
#since this occurs when there is no variance in the effects

## Between Model


In [116]:
#The between estimator is an alternative, usually less efficient estimator, 
#can can be used to estimate model parameters. It is particular simple since 
#it first computes the time averages of y and x and then runs a simple regression 
#using these averages. The year dummies are dropped since the averaging 
#removes differences due to the year.

In [117]:
from linearmodels.panel import BetweenOLS
exog_vars = ['radianceoverpixels', 'summedradiance', 'population', 'radpop_scaled', 'headyears_educated', 'hospital_visits_per_Capita', 'illegal_acts']
exog = sm.add_constant(data[exog_vars])
mod = BetweenOLS(np.log(data.consumption_per_adult_equivalent), exog)
be_res = mod.fit()
print(be_res)

                                BetweenOLS Estimation Summary                                 
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.8153
Estimator:                               BetweenOLS   R-squared (Between):              0.8153
No. Observations:                                11   R-squared (Within):               0.0209
Date:                              Fri, Apr 24 2020   R-squared (Overall):              0.0412
Time:                                      22:15:54   Log-likelihood                    21.652
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      1.8919
Entities:                                        11   P-value                           0.3233
Avg Obs:                                    10.0000   Distribution:                     F(7,3)
Min Obs:                                    10.000

## Including Fixed Effects

In [118]:
#When effects are correlated with the regressors the RE and BE estimators are not consistent. 
#The usual solution is to use Fixed Effects which are available in PanelOLS. 
#Fixed effects are called entity_effects when applied to entities and time_effects 
#when applied to the time dimension:


#### Including fixed effects


In [119]:
from linearmodels.panel import PanelOLS
exog_vars = ['radianceoverpixels', 'summedradiance', 'population', 'radpop_scaled', 'headyears_educated', 'hospital_visits_per_Capita', 'illegal_acts']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(np.log(data.consumption_per_adult_equivalent), exog, entity_effects=False, time_effects=True)
fe_res = mod.fit()

print(fe_res)

                                 PanelOLS Estimation Summary                                  
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.4846
Estimator:                                 PanelOLS   R-squared (Between):              0.7811
No. Observations:                               110   R-squared (Within):               0.0054
Date:                              Fri, Apr 24 2020   R-squared (Overall):              0.0252
Time:                                      22:15:55   Log-likelihood                    126.06
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      12.490
Entities:                                        11   P-value                           0.0000
Avg Obs:                                    10.0000   Distribution:                    F(7,93)
Min Obs:                                    10.000

In [120]:
from linearmodels.panel import PanelOLS
exog_vars = ['radianceoverpixels', 'summedradiance', 'population', 'radpop_scaled', 'headyears_educated', 'hospital_visits_per_Capita', 'illegal_acts']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(np.log(data.consumption_per_adult_equivalent), exog, entity_effects=True)
fe_res = mod.fit()

print(fe_res)



                                 PanelOLS Estimation Summary                                  
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.4828
Estimator:                                 PanelOLS   R-squared (Between):             -641.56
No. Observations:                               110   R-squared (Within):               0.4828
Date:                              Fri, Apr 24 2020   R-squared (Overall):             -15.900
Time:                                      22:15:55   Log-likelihood                   -40.456
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      12.269
Entities:                                        11   P-value                           0.0000
Avg Obs:                                    10.0000   Distribution:                    F(7,92)
Min Obs:                                    10.000

#### Time Effects

In [121]:
from linearmodels.panel import PanelOLS
exog_vars = ['radianceoverpixels', 'summedradiance', 'population', 'radpop_scaled', 'headyears_educated', 'hospital_visits_per_Capita', 'illegal_acts']
exog = sm.add_constant(data[exog_vars])
mod = PanelOLS(np.log(data.consumption_per_adult_equivalent), exog, entity_effects=True, time_effects=True)
fe_te_res = mod.fit()
print(fe_te_res)

                                 PanelOLS Estimation Summary                                  
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.2014
Estimator:                                 PanelOLS   R-squared (Between):             -8.7242
No. Observations:                               110   R-squared (Within):              -0.1534
Date:                              Fri, Apr 24 2020   R-squared (Overall):             -0.3721
Time:                                      22:15:56   Log-likelihood                    144.55
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      2.9899
Entities:                                        11   P-value                           0.0076
Avg Obs:                                    10.0000   Distribution:                    F(7,83)
Min Obs:                                    10.000

#### Using first differences

In [122]:
from linearmodels.panel import FirstDifferenceOLS
exog_vars = ['radianceoverpixels', 'summedradiance', 'population', 'radpop_scaled', 'headyears_educated', 'hospital_visits_per_Capita', 'illegal_acts']
exog = sm.add_constant(data[exog_vars])
exog = data[exog_vars]
mod = FirstDifferenceOLS(np.log(data.consumption_per_adult_equivalent), exog)
fd_res = mod.fit()
print(fd_res)

                            FirstDifferenceOLS Estimation Summary                             
Dep. Variable:     consumption_per_adult_equivalent   R-squared:                        0.4569
Estimator:                       FirstDifferenceOLS   R-squared (Between):             -0.9215
No. Observations:                                99   R-squared (Within):               0.4188
Date:                              Fri, Apr 24 2020   R-squared (Overall):             -0.9188
Time:                                      22:15:58   Log-likelihood                   -44.753
Cov. Estimator:                          Unadjusted                                           
                                                      F-statistic:                      11.055
Entities:                                        11   P-value                           0.0000
Avg Obs:                                    10.0000   Distribution:                    F(7,92)
Min Obs:                                    10.000

In [123]:
#First differencing is an alternative to using fixed effects when there might be correlation. 
#When using first differences, time-invariant variables must be excluded. 
#Additionally, only one linear time-trending variable can be included since this 
#will look like a constant. This variable will soak up all time-trends in the data,
#and so interpretations of these variable can be challenging.

## Comparing models

In [124]:
from linearmodels.panel import compare
print(compare({'BE':be_res,'RE':re_res,'Pooled':pooled_res, }))



                                                           Model Comparison                                                          
                                                             BE                                 RE                             Pooled
-------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable                  consumption_per_adult_equivalent   consumption_per_adult_equivalent   consumption_per_adult_equivalent
Estimator                                            BetweenOLS                      RandomEffects                          PooledOLS
No. Observations                                             11                                110                                110
Cov. Est.                                            Unadjusted                         Unadjusted                         Unadjusted
R-squared                                                0.815

In [125]:
print(compare({'BE':be_res,'RE':re_res,'FE':fe_res, }))

                                                               Model Comparison                                                              
                                                                 BE                                   RE                                   FE
---------------------------------------------------------------------------------------------------------------------------------------------
Dep. Variable                      consumption_per_adult_equivalent     consumption_per_adult_equivalent     consumption_per_adult_equivalent
Estimator                                                BetweenOLS                        RandomEffects                             PanelOLS
No. Observations                                                 11                                  110                                  110
Cov. Est.                                                Unadjusted                           Unadjusted                           Unadjusted
R-squa

In [126]:
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov_params()
    v_B = re.cov_params()
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B))
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval

In [127]:
hausman(fe_res, re_res)

AttributeError: 'PanelEffectsResults' object has no attribute 'cov_params'

In [128]:
#Hausman Test

#data:  log(average_expenditure) ~ log(radianceoverpixels) + log(headyears_educated) +  ...
#chisq = 1.2205, df = 3, p-value = 0.7481
#alternative hypothesis: one model is inconsistent

#As the p value is significant (0.01<0.05), fixed effects model should be used.
